In [38]:
%config Completer.use_jedi = False

import pandas as pd
from sqlalchemy import create_engine
import datetime
import requests
import pymongo
from pprint import pprint
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
crime_incidents_file = "Data/CriminalIncidentsData.csv"
crime_incidents_df = pd.read_csv(crime_incidents_file)
crime_incidents_df.head()

,Year,Year ending,Police Region,Local Government Area,Incidents Recorded,"Rate per 100,000 population"
0,2021,March,1 North West Metro,Banyule,"6,308","4,794.9"
1,2021,March,1 North West Metro,Brimbank,"14,560","7,132.9"
2,2021,March,1 North West Metro,Darebin,"11,513","6,968.1"
3,2021,March,1 North West Metro,Hobsons Bay,"4,469","4,583.9"
4,2021,March,1 North West Metro,Hume,"15,540","6,301.2"


In [3]:
offence_file = "Data/OffenceDivisionData.csv"
offence_df = pd.read_csv(offence_file)
offence_df.head()

,Year,Year ending,Police Service Area,Local Government Area,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,"PSA Rate per 100,000 population","LGA Rate per 100,000 population"
0,2021,March,Ballarat,Ballarat,A Crimes against the person,A10 Homicide and related offences,A10 Homicide and related offences,2,1.7,1.8
1,2021,March,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,143,118.6,126.5
2,2021,March,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,131,108.6,115.9
3,2021,March,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,"A22 Assault police, emergency services or othe...",34,28.2,30.1
4,2021,March,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,279,231.3,246.8


In [4]:
victim_file = "Data/FamilyVictimsData.csv"
victim_df = pd.read_csv(victim_file)
victim_df.head()

,Year,Year ending,AFM Sex,AFM Age Group,AFM Counter,"Rate per 100,000 population"
0,2021,March,Females,00-04 years,242,127.8
1,2021,March,Females,05-09 years,543,266.1
2,2021,March,Females,10-14 years,"1,696",870.1
3,2021,March,Females,15-17 years,"2,262","2,081.5"
4,2021,March,Females,18-19 years,"2,165","2,994.0"


In [5]:
crime_incidents_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 870 entries, 0 to 869
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Year                         870 non-null    int64 
 1   Year ending                  870 non-null    object
 2   Police Region                870 non-null    object
 3   Local Government Area        870 non-null    object
 4   Incidents Recorded           870 non-null    object
 5   Rate per 100,000 population  830 non-null    object
dtypes: int64(1), object(5)
memory usage: 40.9+ KB


In [6]:
offence_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50287 entries, 0 to 50286
Data columns (total 10 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Year                             50287 non-null  int64 
 1   Year ending                      50287 non-null  object
 2   Police Service Area              50287 non-null  object
 3   Local Government Area            50287 non-null  object
 4   Offence Division                 50287 non-null  object
 5   Offence Subdivision              50287 non-null  object
 6   Offence Subgroup                 50287 non-null  object
 7   Incidents Recorded               50287 non-null  object
 8   PSA Rate per 100,000 population  50287 non-null  object
 9   LGA Rate per 100,000 population  50287 non-null  object
dtypes: int64(1), object(9)
memory usage: 3.8+ MB


In [7]:
victim_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Year                         270 non-null    int64 
 1   Year ending                  270 non-null    object
 2   AFM Sex                      270 non-null    object
 3   AFM Age Group                270 non-null    object
 4   AFM Counter                  270 non-null    object
 5   Rate per 100,000 population  270 non-null    object
dtypes: int64(1), object(5)
memory usage: 12.8+ KB


In [9]:
crime_incidents_df_yrfilter = crime_incidents_df.loc[((crime_incidents_df['Year'] >= 2017) & (crime_incidents_df['Year'] <= 2021))]
crime_incidents_df_yrfilter

,Year,Year ending,Police Region,Local Government Area,Incidents Recorded,"Rate per 100,000 population"
0,2021,March,1 North West Metro,Banyule,"6,308","4,794.9"
1,2021,March,1 North West Metro,Brimbank,"14,560","7,132.9"
2,2021,March,1 North West Metro,Darebin,"11,513","6,968.1"
3,2021,March,1 North West Metro,Hobsons Bay,"4,469","4,583.9"
4,2021,March,1 North West Metro,Hume,"15,540","6,301.2"
...,...,...,...,...,...,...
430,2017,March,4 Western,Total,"66,704","6,795.8"
431,2017,March,Justice Institutions and Immigration Facilities,Justice Institutions and Immigration Facilities,"1,125",NaN
432,2017,March,Justice Institutions and Immigration Facilities,Total,"1,125",NaN
433,2017,March,Unincorporated Vic,Unincorporated Vic,126,NaN


In [10]:
crime_incidents_df_clean = crime_incidents_df_yrfilter.loc[((crime_incidents_df_yrfilter['Local Government Area'] != 'Total') &
                                                           (crime_incidents_df_yrfilter['Local Government Area'] != ' Unincorporated Vic') &
                                                           (crime_incidents_df_yrfilter['Local Government Area'] != ' Justice Institutions and Immigration Facilities'))
                                                          ]
crime_incidents_df_clean['Local Government Area'] = crime_incidents_df_clean['Local Government Area'].str.strip()

In [11]:
crime_incidents_df_clean['Incidents Recorded'] = crime_incidents_df_clean['Incidents Recorded'].replace('[,]', '', regex=True).astype(int)

In [12]:
crime_incidents_df_clean['Rate per 100,000 population'] = crime_incidents_df_clean['Rate per 100,000 population'].replace('[,]', '', regex=True).astype(float)

In [13]:
crime_incidents_df_final = crime_incidents_df_clean[['Year', 'Local Government Area', 'Incidents Recorded', 'Rate per 100,000 population']]

In [14]:
crime_incidents_df_final = crime_incidents_df_final.rename(columns={'Year':'year',
                                                                   'Local Government Area':'lga',
                                                                   'Incidents Recorded': 'incidents_recorded',
                                                                   'Rate per 100,000 population': 'rate_per_100k_population'})

In [15]:
crime_incidents_df_sorted_final = crime_incidents_df_final.sort_values(by=['year', 'incidents_recorded'], ascending=False)

In [16]:
crime_incidents_df_sorted_final = crime_incidents_df_sorted_final.reset_index(drop=True)
crime_incidents_df_sorted_final

,year,lga,incidents_recorded,rate_per_100k_population
0,2021,Melbourne,26031,15199.4
1,2021,Casey,16206,4375.5
2,2021,Greater Geelong,15568,5757.7
3,2021,Hume,15540,6301.2
4,2021,Brimbank,14560,7132.9
...,...,...,...,...
390,2017,Alpine,280,2215.5
391,2017,Towong,174,2881.3
392,2017,Buloke,141,2272.4
393,2017,Queenscliffe,135,4562.4


In [17]:
offence_df_yrfilter = offence_df.loc[(offence_df['Year'] >= 2017) & (offence_df['Year'] <= 2021)]
offence_df_yrfilter.tail()

,Year,Year ending,Police Service Area,Local Government Area,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,"PSA Rate per 100,000 population","LGA Rate per 100,000 population"
25258,2017,March,Yarra Ranges,Yarra Ranges,F Other offences,F30 Other government regulatory offences,F33 Liquor and tobacco licensing offences,3,1.9,1.9
25259,2017,March,Yarra Ranges,Yarra Ranges,F Other offences,F30 Other government regulatory offences,F36 Prostitution offences,1,0.6,0.6
25260,2017,March,Yarra Ranges,Yarra Ranges,F Other offences,F90 Miscellaneous offences,F91 Environmental offences,2,1.3,1.3
25261,2017,March,Yarra Ranges,Yarra Ranges,F Other offences,F90 Miscellaneous offences,F93 Cruelty to animals,2,1.3,1.3
25262,2017,March,Yarra Ranges,Yarra Ranges,F Other offences,F90 Miscellaneous offences,F94 Dangerous substance offences,1,0.6,0.6


In [18]:
offence_df_yrfilter['Incidents Recorded'] = offence_df_yrfilter['Incidents Recorded'].replace('[\,]', '', regex=True).astype(int)
offence_df_yrfilter['LGA Rate per 100,000 population'] = offence_df_yrfilter['LGA Rate per 100,000 population'].replace('[\,]', '', regex=True).astype(float)


In [19]:
offence_df_clean = offence_df_yrfilter[['Year', 'Offence Division', 'Incidents Recorded', 'LGA Rate per 100,000 population']]
offence_df_clean.head()

,Year,Offence Division,Incidents Recorded,"LGA Rate per 100,000 population"
0,2021,A Crimes against the person,2,1.8
1,2021,A Crimes against the person,143,126.5
2,2021,A Crimes against the person,131,115.9
3,2021,A Crimes against the person,34,30.1
4,2021,A Crimes against the person,279,246.8


In [21]:
grouped_offence_df = offence_df_clean.groupby(['Year', 'Offence Division']).agg(
             Incidents=('Incidents Recorded','sum'),
             RatePer100k=('LGA Rate per 100,000 population', lambda x: x.sum() / x.count()))
grouped_offence_df['RatePer100k'] = grouped_offence_df['RatePer100k'].map("{:.2f}".format)
grouped_offence_df = grouped_offence_df.reset_index()
grouped_offence_df.head()

,Year,Offence Division,Incidents,RatePer100k
0,2017,A Crimes against the person,60636,56.61
1,2017,B Property and deception offences,265881,190.15
2,2017,C Drug offences,15657,54.77
3,2017,D Public order and security offences,24419,37.42
4,2017,E Justice procedures offences,47663,101.62


In [22]:
grouped_offence_df_final = grouped_offence_df.rename(columns={'Year':'year', 'Offence Division': 'offence_division',
                                                             'Incidents':'incidents', 'RatePer100k': 'rate_per_100k'})
grouped_offence_df_final.head()

,year,offence_division,incidents,rate_per_100k
0,2017,A Crimes against the person,60636,56.61
1,2017,B Property and deception offences,265881,190.15
2,2017,C Drug offences,15657,54.77
3,2017,D Public order and security offences,24419,37.42
4,2017,E Justice procedures offences,47663,101.62


In [23]:
victim_df_clean_gender = victim_df.loc[victim_df['AFM Sex'] != 'People']
victim_df_clean = victim_df_clean_gender.loc[victim_df_clean_gender['AFM Age Group'] != 'Total']
victim_df_clean['AFM Counter'] = victim_df_clean['AFM Counter'].replace('[,]', '', regex=True).astype(int)
victim_df_clean_final = victim_df_clean[['Year', 'AFM Sex', 'AFM Age Group', 'AFM Counter']]
victim_df_clean_final.head()

,Year,AFM Sex,AFM Age Group,AFM Counter
0,2021,Females,00-04 years,242
1,2021,Females,05-09 years,543
2,2021,Females,10-14 years,1696
3,2021,Females,15-17 years,2262
4,2021,Females,18-19 years,2165


In [24]:
grouped_victim_df = victim_df_clean_final.groupby(['Year','AFM Age Group','AFM Sex']).sum()
grouped_victim_df = grouped_victim_df.reset_index()
grouped_victim_df_final = grouped_victim_df.rename(columns={'Year': 'year',
                                                           'AFM Age Group': 'afm_age_group',
                                                           'AFM Sex': 'afm_gender',
                                                           'AFM Counter': 'afm_counter'})
grouped_victim_df_final.head()

,year,afm_age_group,afm_gender,afm_counter
0,2017,00-04 years,Females,319
1,2017,00-04 years,Males,362
2,2017,05-09 years,Females,683
3,2017,05-09 years,Males,877
4,2017,10-14 years,Females,1689


In [61]:
# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
client.drop_database('crime_database')
# Define the 'classDB' database in Mongo
db = client.crime_database

In [62]:
collection_crime = db['crime_table']
collection_victim = db['victim_table']
collection_offence = db['offence_table']
collection_lga_crime = db['lga_crime_table']

In [63]:
crime_incidents_dict = crime_incidents_df_sorted_final.to_dict("records")

In [64]:
collection_crime.insert_many(crime_incidents_dict)

In [65]:
grouped_victim_dict = grouped_victim_df_final.to_dict("records")

In [66]:
collection_victim.insert_many(grouped_victim_dict)

In [67]:
grouped_offence_dict = grouped_offence_df_final.to_dict("records")

In [68]:
collection_offence.insert_many(grouped_offence_dict)

In [35]:
geojson_url = "https://data.gov.au/geoserver/vic-local-government-areas-psma-administrative-boundaries/wfs?request=GetFeature&typeName=ckan_bdf92691_c6fe_42b9_a0e2_a4cd716fa811&outputFormat=json"
response = requests.get(geojson_url)
response_data = response.json()

In [55]:
for crime_data in crime_incidents_dict:
    i = 0
    for feature in response_data['features']:
        lga_crime_data = {}
        if(feature['properties']['vic_lga__3'] == crime_data['lga'].upper()):
            lga_crime_data['year'] = crime_data['year']
            lga_crime_data['incidents_recorded'] = crime_data['incidents_recorded']
            lga_crime_data['rate_per_100k'] = crime_data['rate_per_100k_population']
            #feature['properties']['lga_crime_data'] = lga_crime_data
            response_data['features'][i]['properties']['lga_crime_data'] = lga_crime_data
        i += 1

In [69]:
collection_lga_crime.insert_many(response_data['features'])